![image.png](attachment:image.png)

##### **Good luck with taking your exam. Keep working and make your dreams all come true. Seeing the results of all of your hard work will make this struggle worth it. We’re all thinking of you.** 
<b><font color='blue'>AI-PRO Spark Team ITI</font></b>

# NLP Using PySpark

## Objective:
- The objective from this project is to create a <b>Spam filter using NaiveBayes classifier</b>.
- It is required to obtain <b>f1_scored > 0.9</b>.
- We'll use a dataset from UCI Repository. SMS Spam Detection: https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection
- Data is also provided for you in the assignment (you do not have to download it).

## To perform this task follow the following guiding steps:

### Create a spark session and import the required libraries

In [2]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 47 kB/s 
     |████████████████████████████████| 199 kB 41.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=558deaa7cd53cfe74aaf48cc13d0b13b687b84c7035c1fc44d44bf47b8449ca0
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [3]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

### Read the readme file to learn more about the data

### Read the data into a DataFrame

In [19]:
df = spark.read.csv('SMSSpamCollection.txt',sep='\t',inferSchema=True)

### Print the schema

In [20]:
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)



In [23]:
df.show(5)

+----+--------------------+
| _c0|                 _c1|
+----+--------------------+
| ham|Go until jurong p...|
| ham|Ok lar... Joking ...|
|spam|Free entry in 2 a...|
| ham|U dun say so earl...|
| ham|Nah I don't think...|
+----+--------------------+
only showing top 5 rows



### Rename the first column to 'class' and second column to 'text'

In [24]:
df2=df.withColumnRenamed("_c0","class").withColumnRenamed("_c1","text")

In [25]:
df2.show(3)

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
+-----+--------------------+
only showing top 3 rows



### Show the first 10 rows from the dataframe
- Show once with truncate=True and once with truncate=False

In [26]:
df2.show(10,truncate=False)

+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------+
|class|text                                                                                                                                                            |
+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------+
|ham  |Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...                                                 |
|ham  |Ok lar... Joking wif u oni...                                                                                                                                   |
|spam |Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075o

In [27]:
df2.show(10)

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
+-----+--------------------+
only showing top 10 rows



## Clean and Prepare the Data

### Create a new feature column contains the length of the text column

In [29]:
import pyspark.sql.functions as F
df3=df2.withColumn('lengthoftext', F.length('text'))

In [30]:
df3.show()

+-----+--------------------+------------+
|class|                text|lengthoftext|
+-----+--------------------+------------+
|  ham|Go until jurong p...|         111|
|  ham|Ok lar... Joking ...|          29|
| spam|Free entry in 2 a...|         155|
|  ham|U dun say so earl...|          49|
|  ham|Nah I don't think...|          61|
| spam|FreeMsg Hey there...|         147|
|  ham|Even my brother i...|          77|
|  ham|As per your reque...|         160|
| spam|WINNER!! As a val...|         157|
| spam|Had your mobile 1...|         154|
|  ham|I'm gonna be home...|         109|
| spam|SIX chances to wi...|         136|
| spam|URGENT! You have ...|         155|
|  ham|I've been searchi...|         196|
|  ham|I HAVE A DATE ON ...|          35|
| spam|XXXMobileMovieClu...|         149|
|  ham|Oh k...i'm watchi...|          26|
|  ham|Eh u remember how...|          81|
|  ham|Fine if thats th...|          56|
| spam|England v Macedon...|         155|
+-----+--------------------+------

### Get the average text length for each class (give alias name to the average length column)

In [31]:
df3.groupBy("class").avg("lengthoftext").show(truncate=False)

+-----+-----------------+
|class|avg(lengthoftext)|
+-----+-----------------+
|ham  |71.45431945307645|
|spam |138.6706827309237|
+-----+-----------------+



## Feature Transformations

### In this part you transform you raw text in to tf_idf model :
- For more information about TF-IDF check the following link: <b>(Not needed for the test)</b>
https://en.wikipedia.org/wiki/Tf%E2%80%93idf

### Perform the following steps to obtain TF-IDF:
1. Import the required transformers/estimators for the subsequent steps.
2. Create a <b>Tokenizer</b> from the text column.
3. Create a <b>StopWordsRemover</b> to remove the <b>stop words</b> from the column obtained from the <b>Tokenizer</b>.
4. Create a <b>CountVectorizer</b> after removing the <b>stop words</b>.
5. Create the <b>TF-IDF</b> from the <b>CountVectorizer</b>.

In [39]:
from pyspark.ml.feature import *

In [40]:
tokenizer = Tokenizer(outputCol="words")
tokenizer.setInputCol("text")

df4=tokenizer.transform(df3)
df4.show(3,truncate=False)

+-----+-----------------------------------------------------------------------------------------------------------------------------------------------------------+------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|class|text                                                                                                                                                       |lengthoftext|words                                                                                                                                                                                   |
+-----+-----------------------------------------------------------------------------------------------------------------------------------------------------------+------------+----------------------------------------------------------------------------------------------------

In [47]:
#any some stop words
remover = StopWordsRemover(stopWords=["a","an"])

remover.setInputCol("words")
remover.setOutputCol("stopwords")


df5=remover.transform(df4)
df5.show(3)

+-----+--------------------+------------+--------------------+--------------------+
|class|                text|lengthoftext|               words|           stopwords|
+-----+--------------------+------------+--------------------+--------------------+
|  ham|Go until jurong p...|         111|[go, until, juron...|[go, until, juron...|
|  ham|Ok lar... Joking ...|          29|[ok, lar..., joki...|[ok, lar..., joki...|
| spam|Free entry in 2 a...|         155|[free, entry, in,...|[free, entry, in,...|
+-----+--------------------+------------+--------------------+--------------------+
only showing top 3 rows



In [50]:
cv = CountVectorizer()
cv.setInputCol("stopwords")
cv.setOutputCol("CV")

model = cv.fit(df5)

model.setInputCol("stopwords")
df6=model.transform(df5)
df6.show(3,truncate=False)

+-----+-----------------------------------------------------------------------------------------------------------------------------------------------------------+------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|class|text                                                                                                                                                       |lengthoftext|words                                                                              

In [54]:
idf=IDF(inputCol="CV", outputCol="tf-idf_features")
idfModel = idf.fit(df6)
df7= idfModel.transform(df6)
df7.show(3,truncate=False)

+-----+-----------------------------------------------------------------------------------------------------------------------------------------------------------+------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

- Convert the <b>class column</b> to index using <b>StringIndexer</b>
- Create feature column from the <b>TF-IDF</b> and <b>lenght</b> columns.

In [67]:
stringIndexer = StringIndexer(inputCol="class", outputCol="label")

model = stringIndexer.fit(df7)


df8 = model.transform(df7)
df8.show(3,truncate=False)

+-----+-----------------------------------------------------------------------------------------------------------------------------------------------------------+------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [69]:
assembler = VectorAssembler(
    inputCols=["lengthoftext","tf-idf_features"],
    outputCol='features')


df9=assembler.transform(df8)

df9.show(6,truncate=False)

+-----+-----------------------------------------------------------------------------------------------------------------------------------------------------------+------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## The Model
- Create a <b>NaiveBayes</b> classifier with the default parameters.

In [70]:
from pyspark.ml.classification import NaiveBayes 
nb = NaiveBayes(modelType="multinomial")

In [71]:
nbmodel = nb.fit(df9)

In [73]:
#from pyspark.ml.evaluation import MulticlassClassificationEvaluator
#evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="pr ediction", metricName="accuracy") 
#nbaccuracy = evaluator.evaluate(df9) 
#print("Test accuracy = " + str(nbaccuracy))

## Pipeline
### Create a pipeline model contains all the steps starting from the Tokenizer to the NaiveBays classifier.

In [74]:
from pyspark.ml import Pipeline 

indexers = [tokenizer,remover,cv,idf,stringIndexer,assembler,nb ]
pipeline = Pipeline(stages=indexers)

### Split your data to trian and test data with ratios 0.7 and 0.3 respectively.

In [75]:
train , test = df3.randomSplit([0.7,0.3], 42) 

### Fit your Pipeline model to the training data

In [77]:
final_model=pipeline.fit(train)

### Perform predictions on tests dataframe

In [80]:
final_test=final_model.transform(test)

### Print the schema of the prediction dataframe

In [81]:
final_test.printSchema()

root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)
 |-- lengthoftext: integer (nullable = true)
 |-- words: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- stopwords: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- CV: vector (nullable = true)
 |-- tf-idf_features: vector (nullable = true)
 |-- label: double (nullable = false)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



## Model Evaluation
- Use <b>MulticlassClassificationEvaluator</b> to calculate the <b>f1_score</b>.

In [83]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1") 
nbaccuracy = evaluator.evaluate(final_test) 
print("Test accuracy = " + str(nbaccuracy))

Test accuracy = 0.9737387349839617


f1_score is: 0.9664707489549014


# GOOD LUCK
<b><font color='GREEN'>AI-PRO Spark Team ITI</font></b>

![image-3.png](attachment:image-3.png)